In [2]:
import pandas as pd
import tensorflow as tf

# Automatically reload imported modules that are changed outside this notebook
# More pixels in figures
import matplotlib.pyplot as plt
plt.rcParams["figure.dpi"] = 200


In [3]:
import numpy as np
np_rng = np.random.default_rng(1)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))



import urllib.parse
from IPython.display import display, Markdown

import os

from lidbox.meta import (
    common_voice,
    generate_label2target,
    verify_integrity,
    read_audio_durations,
    random_oversampling_on_split
)

tf.random.set_seed(np_rng.integers(0, tf.int64.max))

train = pd.read_csv("train.tsv", sep="\t")
test = pd.read_csv("test.tsv", sep="\t")
dev = pd.read_csv("dev.tsv", sep="\t")

train["path"] = train["path"].apply(lambda x: x[:-3] + "mp3")
test["path"] = test["path"].apply(lambda x: x[:-3] + "mp3")
dev["path"] = dev["path"].apply(lambda x: x[:-3] + "mp3")

train["split"] = "train"
test["split"] = "test"
dev["split"] = "dev"
test = test.sample(30000, replace=False)
meta = pd.concat([train, test, dev])


In [4]:
meta.loc[meta["locale"] != "kz", "path"] = "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/" +  meta.loc[meta["locale"] != "kz"]["locale"] + "/clips/" + meta.loc[meta["locale"] != "kz"]["path"]
targets = {"kz": 0, "ru": 1, "en":2, "other":3}
meta["target"] = meta["locale"]
meta.loc[(meta["locale"] != "kz") & (meta["locale"] != "ru") & (meta["locale"]!="en"), "target"] = "other"
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5f590a130a73c.mp3"]
meta = meta.loc[meta["path"] != "/tf/datasets/data_untar/cv-corpus-6.1-2020-12-11/kz/clips/5ef9bd9ba7029.mp3"]

meta["id"] = meta["Unnamed: 0"].apply(str)
meta["target"] = meta["target"].map(targets)

workdir = "/tf/datasets/x-vector"


#meta["target"] = meta["target"].apply(to_1hot)


2021-06-02 03:26:09.264 I numexpr.utils: Note: detected 96 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.
2021-06-02 03:26:09.265 I numexpr.utils: Note: NumExpr detected 96 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
2021-06-02 03:26:09.265 I numexpr.utils: NumExpr defaulting to 8 threads.


In [5]:
import scipy.signal

from lidbox.features import audio, cmvn
import lidbox.data.steps as ds_steps


TF_AUTOTUNE = tf.data.experimental.AUTOTUNE


def metadata_to_dataset_input(meta):
    return {
        "id": tf.constant(meta.id, tf.string),
        "path": tf.constant(meta.path, tf.string),
        "target": tf.constant(meta.target, tf.int32),
        "split": tf.constant(meta.split, tf.string),
    }

def read_mp3(x):
    s, r = audio.read_mp3(x["path"])
    out_rate = 16000
    s = audio.resample(s, r, out_rate)
    s = audio.peak_normalize(s, dBFS=-3.0)
    s = audio.remove_silence(s, out_rate)
    return dict(x, signal=s, sample_rate=out_rate)


def random_filter(x):
    def scipy_filter(s, N=10):
        b = np_rng.normal(0, 1, N)
        return scipy.signal.lfilter(b, 1.0, s).astype(np.float32), b
    s, _ = tf.numpy_function(
        scipy_filter,
        [x["signal"]],
        [tf.float32, tf.float64],
        name="np_random_filter")
    s = tf.cast(s, tf.float32)
    s = audio.peak_normalize(s, dBFS=-3.0)
    return dict(x, signal=s)


def random_speed_change(ds):
    return ds_steps.random_signal_speed_change(ds, min=0.9, max=1.1, flag=None)


def create_signal_chunks(ds):
    ds = ds_steps.repeat_too_short_signals(ds, 3200)
    ds = ds_steps.create_signal_chunks(ds, 3200, 800)
    return ds


def batch_extract_features(x):
    with tf.device("GPU"):
        signals, rates = x["signal"], x["sample_rate"]
        S = audio.spectrograms(signals, rates[0])
        S = audio.linear_to_mel(S, rates[0])
        S = tf.math.log(S + 1e-6)
        S = cmvn(S, normalize_variance=False)
    return dict(x, logmelspec=S)

def pipeline_from_meta(data, split):
    if split == "train":
        data = data.sample(frac=1, random_state=np_rng.bit_generator)

    ds = (tf.data.Dataset
            .from_tensor_slices(metadata_to_dataset_input(data))
            .map(read_mp3, num_parallel_calls=TF_AUTOTUNE))

    if split == "train":
        return (ds
            .apply(random_speed_change)
           #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1)
            .map(random_filter, num_parallel_calls=TF_AUTOTUNE)
            .apply(create_signal_chunks)
            .batch(1)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch())
    else:
        return (ds
            .apply(create_signal_chunks)
            .batch(1)
            .map(batch_extract_features, num_parallel_calls=TF_AUTOTUNE)
            .unbatch()
            #.cache(os.path.join(cachedir, "data", split))
            .prefetch(1))


cachedir = os.path.join(workdir, "cache")

split2ds = {split: pipeline_from_meta(meta[meta["split"]==split], split)
            for split in meta.split.unique()}

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2021-06-02 03:26:13.192 W tensorflow: From /root/anaconda3/lib/python3.7/site-packages/tensorflow/python/ops/array_ops.py:5049: calling gather (from tensorflow.python.ops.array_ops) with validate_indices is deprecated and will be removed in a future version.
Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.


2021-06-02 03:26:13.338 I lidbox.data.steps: Applying random resampling to signals with a random speed ratio chosen uniformly at random from [0.900, 1.100]
2021-06-02 03:26:13.558 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-02 03:26:13.644 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-02 03:26:15.345 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-02 03:26:15.357 I lidbox.data.steps: Dividing every signal in the dataset into new signals by creating signal chunks of length 3200 ms and offset 800 ms. Maximum amount of padding allowed in the last chunk is 0 ms.
2021-06-02 03:26:15.792 I lidbox.data.steps: Repeating all signals until they are at least 3200 ms
2021-06-02 03:26:15.805 I lidbox.data.steps: Dividing every signal in the dataset into new signals by

In [9]:
import lidbox.models.xvector as xvector
model_input_type = "logmelspec"

def create_model(num_freq_bins, num_labels):
    model = xvector.create([None, num_freq_bins], num_labels, channel_dropout_rate=0.8)
    model.compile(
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5), 
        metrics=tf.keras.metrics.sparse_categorical_accuracy)
    return model


model = create_model(
    num_freq_bins=20 if model_input_type == "mfcc" else 40,
    num_labels=4)
model.summary()

Model: "x-vector"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, None, 40)]        0         
_________________________________________________________________
channel_dropout (SpatialDrop (None, None, 40)          0         
_________________________________________________________________
frame1 (Conv1D)              (None, None, 512)         102912    
_________________________________________________________________
frame2 (Conv1D)              (None, None, 512)         786944    
_________________________________________________________________
frame3 (Conv1D)              (None, None, 512)         786944    
_________________________________________________________________
frame4 (Conv1D)              (None, None, 512)         262656    
_________________________________________________________________
frame5 (Conv1D)              (None, None, 1500)        769

In [10]:
callbacks = [
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(cachedir, "tensorboard", model.name),
        update_freq="epoch",
        write_images=True,
        profile_batch=0,
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
    ),
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(cachedir, "model", model.name),
        monitor='val_loss',
        save_weights_only=True,
        save_best_only=True,
        verbose=1,
    ),
]


def as_model_input(x):
    return x["logmelspec"], x["target"]



In [ ]:
print("preparing datasets")

train_ds = split2ds["train"].map(as_model_input)
dev_ds = split2ds["dev"].map(as_model_input)

 
print("start training")    
with tf.device("GPU"):
    history = model.fit(
        train_ds.batch(32).repeat(100),
        steps_per_epoch=2576,
        validation_data=dev_ds.batch(32).repeat(100),
        validation_steps=961,
        callbacks=callbacks,
        verbose=1,
        epochs=100)

tf.keras.models.save_model(
    model, "xvector.h5", overwrite=True, include_optimizer=True, save_format=None,
    signatures=None, options=None, save_traces=True
)